Table 4.10.1: Injecting status of new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.10.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Injecting Status')
cell.assert_one()
obs =  tab.filter('Missing/inconsistent').shift(0,1).expand(RIGHT)
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace() - obs
InjectingStatus = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(InjectingStatus,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','new-presentations-to-treatment'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'].unique()

array(['Never injected', 'Previously injected', 'Currently injecting',
       'Declined to answer', 'Missing/inconsistent', 'Total'],
      dtype=object)

In [9]:
new_table['Basis of treatment'] = new_table['Treatment group'].map(
    lambda x: {
        'Total' : 'injecting-status/total',
        'Never injected': 'injecting-status/never-injected',
        'Previously injected' : 'injecting-status/previously-injected',
        'Currently injecting': 'injecting-status/currently-injecting',
        'Declined to answer' : 'injecting-status/declined-to-answer',
        'Missing/inconsistent' : 'injecting-status/missing/inconsistent',
        }.get(x, x))

In [10]:
import urllib.request as request
import csv
import io
import requests
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [11]:
new_table

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/never-injected,opiate,Count,16165,clients-in-treatment
1,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/never-injected,non-opiate-only,Count,14289,clients-in-treatment
2,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/never-injected,non-opiate-and-alcohol,Count,16263,clients-in-treatment
3,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/never-injected,alcohol-only,Count,48501,clients-in-treatment
4,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/never-injected,total,Count,95218,clients-in-treatment
5,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/previously-injected,opiate,Count,14022,clients-in-treatment
6,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/previously-injected,non-opiate-only,Count,1092,clients-in-treatment
7,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/previously-injected,non-opiate-and-alcohol,Count,1407,clients-in-treatment
8,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/previously-injected,alcohol-only,Count,1303,clients-in-treatment
9,gregorian-interval/2017-04-01T00:00:00/P1Y,injecting-status/previously-injected,total,Count,17824,clients-in-treatment
